### How to use few shot examples in chat models
https://python.langchain.com/docs/how_to/few_shot_examples_chat/

질문과 답변의 쌍을 여러개 예시로 만들어, 질의할 때 같이 포함하여 답변의 정확도를 크게 끌어올린다.

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [3]:
llm = ChatOllama(
    model="gemma2:9b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
    # other params...
)

In [4]:
ai_msg = llm.invoke("What is 2 🍎 9?")

In [5]:
ai_msg.content

"This looks like a fun math problem with a twist!  \n\nSince we're dealing with apples, it's likely not standard addition. Here are a couple of ways to think about it:\n\n* **Counting Apples:** If you have 2 apples and then add 9 more, you would have a total of 11 apples. So, 2 🍎 9 = 11 🍎\n\nLet me know if you had something else in mind!  \n"

In [6]:
examples = [
    {"input": "2 🍎 2", "output": "4"},
    {"input": "2 🍎 3", "output": "5"}
]

In [7]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

In [8]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

In [9]:
few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': '2 🍎 2', 'output': '4'}, {'input': '2 🍎 3', 'output': '5'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})]))

In [10]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

In [11]:
final_prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a wondrous wizard of math.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '2 🍎 2', 'output': '4'}, {'input': '2 🍎 3', 'output': '5'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, temp

In [12]:
chain = final_prompt | llm

In [13]:
ai_msg2 = chain.invoke({"input": "What is 9 🍎 2?"})

In [14]:
ai_msg2.content

'9 🍎 2 = 11 \n\nRemember, when we use "🍎" it means we\'re adding the numbers together!  😊  \n'

### Dynamic few-shot prompting

여러가지 질의-답변 쌍중에 관련있는 예제만 k 개 추출하여 모델에 넣어 답변의 정확도를 높이는 방법.

In [1]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [3]:
llm = ChatOllama(
    model="gemma2:9b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
    # other params...
)

In [4]:
examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
    {"input": "2 🦜 4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

In [5]:
to_vectorize = [" ".join(example.values()) for example in examples]

In [6]:
to_vectorize

['2 🦜 2 4',
 '2 🦜 3 5',
 '2 🦜 4 6',
 'What did the cow say to the moon? nothing at all',
 'Write me a poem about the moon One for the moon, and one for me, who are we to talk about the moon?']

In [7]:
embeddings = OllamaEmbeddings(model="gemma2:9b", 
                             base_url="http://host.docker.internal:11434")

In [8]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [9]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

In [10]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore, k=2
)

In [11]:
example_selector.select_examples({"input": "horse"})

[{'input': 'Write me a poem about the moon',
  'output': 'One for the moon, and one for me, who are we to talk about the moon?'},
 {'input': 'What did the cow say to the moon?', 'output': 'nothing at all'}]

3 🦜 3 이라는 인풋값을 넣으면 그에 맞는 예시를 추출하게 된다. 사실 수많은 예시중에 모델에 넣을 예시를 선택하는 것도 LLM 을 사용하게 될줄은 몰랐다.

In [12]:
example_selector.select_examples({"input": "3 🦜 3"})

[{'input': '2 🦜 2', 'output': '4'}, {'input': '2 🦜 3', 'output': '5'}]

### Create prompt template

이제 실제 질의할 프롬프트를 만들어 실행해보자.

In [13]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [14]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    )
)

In [15]:
few_shot_prompt.invoke(input="What's 3 🦜 10?").to_messages()

[HumanMessage(content='2 🦜 4', additional_kwargs={}, response_metadata={}),
 AIMessage(content='6', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='5', additional_kwargs={}, response_metadata={})]

In [16]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

In [21]:
final_prompt.invoke(input="What's 3 🦜 13?").to_messages()

[SystemMessage(content='You are a wondrous wizard of math.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='2 🦜 4', additional_kwargs={}, response_metadata={}),
 AIMessage(content='6', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='5', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's 3 🦜 13?", additional_kwargs={}, response_metadata={})]

In [18]:
chain = final_prompt | llm

In [19]:
chain.invoke({"input": "What's 3 🦜 13?"})

AIMessage(content='This looks like a fun code!  \n\n"🦜" seems to mean "plus". So, 3 🦜 13 is the same as 3 + 13 = 16. \n\n\nLet me know if you have more math puzzles for me! 😊 \n', additional_kwargs={}, response_metadata={'model': 'gemma2:9b', 'created_at': '2024-10-08T15:12:49.509617Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6663140500, 'load_duration': 54041667, 'prompt_eval_count': 61, 'prompt_eval_duration': 110685000, 'eval_count': 60, 'eval_duration': 6490037000}, id='run-1d5e1324-f82a-456d-a28c-1ea7c25eb159-0', usage_metadata={'input_tokens': 61, 'output_tokens': 60, 'total_tokens': 121})